In [1]:
!pip install --upgrade transformers einops datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 11.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 60.3 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.12.0
    Uninstalling datasets-2.12.0:
      Successfully uninstalled datasets-2.12.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.0.dev0
    Uninstalling accelerate-0.20.0.dev0:
      Successfully uninstalled accelerate-0.20.0.dev0


Import The Dependencies

In [3]:
import torch
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from typing import Any, Dict, Tuple
import warnings
from threading import Event, Thread
import textwrap
from datasets import concatenate_datasets


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda118.so...


/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/workspace/Untitled.ipynb')}
  warn(msg)
/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/cuda_set

Mosaic Instruct Model MPT-7B-Instruct

In [4]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


class InstructionTextGenerationPipeline:
    def __init__(
        self,
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right" # "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.5,
            "top_p": 0.92,
            "top_k": 0,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.1,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def format_instruction(self, instruction):
        return PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        s = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)
        input_ids = self.tokenizer(s, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # Slice the output_ids tensor to get only new tokens
        new_tokens = output_ids[0, len(input_ids[0]) :]
        output_text = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
        return output_text

In [5]:
generate = InstructionTextGenerationPipeline(
    "mosaicml/mpt-7b-instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- param_init_fns.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- attention.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- blocks.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- modeling_mpt.py
- meta_init_context.py
- custom_embedding.py
- param_init_fns.py
- hf_prefixlm_converter.py
- blocks.py
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/1fc4634127ec64a45716003578b9cfae23265849/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_2808/1407196826.py:38: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


In [6]:
 # Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def process_stream(instruction, temperature, top_p, top_k, max_new_tokens):
    # Tokenize the input
    input_ids = generate.tokenizer(
        generate.format_instruction(instruction), return_tensors="pt"
    ).input_ids
    input_ids = input_ids.to(generate.model.device)

    # Initialize the streamer and stopping criteria
    streamer = TextIteratorStreamer(
        generate.tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )
    stop = StopOnTokens()

    if temperature < 0.1:
        temperature = 0.0
        do_sample = False
    else:
        do_sample = True

    gkw = {
        **generate.generate_kwargs,
        **{
            "input_ids": input_ids,
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": do_sample,
            "top_p": top_p,
            "top_k": top_k,
            "streamer": streamer,
            "stopping_criteria": StoppingCriteriaList([stop]),
        },
    }

    response = ''

    def generate_and_signal_complete():
        generate.model.generate(**gkw)

    t1 = Thread(target=generate_and_signal_complete)
    t1.start()

    for new_text in streamer:
        response += new_text
   
    return response

In [7]:
def generate_text(instruction, temperature = 0.3, top_p = 0.95, top_k = 0, max_new_tokens = 2000):
  response = process_stream(instruction, temperature, top_p, top_k, max_new_tokens)

  wrapped_text = textwrap.fill(response, width=100)
  print(wrapped_text)

In [8]:
generate_text('Hello')

Hi


Process the Data

In [9]:
dataset = load_dataset('ag_news')
# Extract label names
label_names = dataset['train'].features['label'].names
label_names

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

['World', 'Sports', 'Business', 'Sci/Tech']

In [10]:
label_names_str = str(label_names).replace("'","")
prefix = f"Classify the following text into one of the following categories: {label_names_str}\nText: "

num_samples_per_class = 75  # Set this to determine the number of samples per class

# Extract subsets of the training dataset with equal number of samples per class
train_datasets = []

for label in range(len(label_names)):
    label_dataset = dataset['train'].filter(lambda x: x['label'] == label).select(range(num_samples_per_class))
    label_dataset = label_dataset.map(lambda x: {'text': generate.format_instruction(prefix + x['text']) + label_names[x['label']]})
    train_datasets.append(label_dataset)

train_dataset = concatenate_datasets(train_datasets)

# Do the same for the validation dataset, if needed
val_datasets = []
for label in range(len(label_names)):
    label_dataset = dataset['test'].filter(lambda x: x['label'] == label).select(range(num_samples_per_class))
    label_dataset = label_dataset.map(lambda x: {'text': generate.format_instruction(prefix + x['text']) + label_names[x['label']]})
    val_datasets.append(label_dataset)

val_dataset = concatenate_datasets(val_datasets)

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Parameter 'function'=<function <lambda> at 0x7f8f3fee70d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

In [11]:
# Tokenize the datasets
train_encodings = generate.tokenizer(train_dataset['text'], truncation=True, padding=True, max_length=256)
val_encodings = generate.tokenizer(val_dataset['text'], truncation=True, padding=True, max_length=256)

In [12]:
# Convert the encodings to PyTorch datasets
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [13]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)
val_dataset = TextDataset(val_encodings)

Before Finetuning, Let's see the output

In [15]:
index = 0
example_text = prefix + dataset['test'][index]['text'] 
print('Example text:')
print(textwrap.fill(example_text, width=100))
print("Response:")
generate_text(example_text, temperature=0)
print("Correct Label:")
print(label_names[dataset['test'][index]['label']])

Example text:
Classify the following text into one of the following categories: [World, Sports, Business,
Sci/Tech] Text: Fears for T N pension after talks Unions representing workers at Turner   Newall say
they are 'disappointed' after talks with stricken parent firm Federal Mogul.
Response:
The above paragraph can be classified as follows; World - Disappointment from unions over failed
negotiations between their employer and its owner (Turner Newall).
Correct Label:
Business


Freeze the Layers

In [16]:
for name, param in generate.model.named_parameters():
    print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
    if "31" not in name:
        param.requires_grad = False
    print(name, param.requires_grad)

transformer.wte.weight   Modelsize: 206.6M parameters
transformer.wte.weight False
transformer.blocks.0.norm_1.weight   Modelsize: 0.0M parameters
transformer.blocks.0.norm_1.weight False
transformer.blocks.0.attn.Wqkv.weight   Modelsize: 50.3M parameters
transformer.blocks.0.attn.Wqkv.weight False
transformer.blocks.0.attn.out_proj.weight   Modelsize: 16.8M parameters
transformer.blocks.0.attn.out_proj.weight False
transformer.blocks.0.norm_2.weight   Modelsize: 0.0M parameters
transformer.blocks.0.norm_2.weight False
transformer.blocks.0.ffn.up_proj.weight   Modelsize: 67.1M parameters
transformer.blocks.0.ffn.up_proj.weight False
transformer.blocks.0.ffn.down_proj.weight   Modelsize: 67.1M parameters
transformer.blocks.0.ffn.down_proj.weight False
transformer.blocks.1.norm_1.weight   Modelsize: 0.0M parameters
transformer.blocks.1.norm_1.weight False
transformer.blocks.1.attn.Wqkv.weight   Modelsize: 50.3M parameters
transformer.blocks.1.attn.Wqkv.weight False
transformer.blocks.1.a

In [17]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=50,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,              
    logging_dir='./logs',            
    logging_steps=10,
)


In [18]:
# Create the Trainer and train
trainer = Trainer(
    model=generate.model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=val_dataset             
)

In [19]:
trainer.train()

/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR API key must be 40 characters long, yours was 7
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Step,Training Loss
10,3.032800
20,2.946900
30,2.901600
40,2.825000
50,2.634400
60,2.268700
70,2.103100
80,1.899200
90,1.684400
100,1.483600


TrainOutput(global_step=950, training_loss=0.9295086348684211, metrics={'train_runtime': 677.3493, 'train_samples_per_second': 22.145, 'train_steps_per_second': 1.403, 'total_flos': 9.509450563584e+16, 'train_loss': 0.9295086348684211, 'epoch': 50.0})

In [20]:
generate.model.eval()

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

LETS TEST

In [27]:
index = 4
example_text = prefix + dataset['test'][index]['text'] 
print('Example text:')
print(textwrap.fill(example_text, width=100))
print("Response:")
generate_text(example_text, temperature=0)
print("Correct Label:")
print(label_names[dataset['test'][index]['label']])

Example text:
Classify the following text into one of the following categories: [World, Sports, Business,
Sci/Tech] Text: Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting
agency went after emissions of the bovine variety Friday, adopting the nation's first rules to
reduce air pollution from dairy cow manure.
Response:
Sci/Tech
Correct Label:
Sci/Tech
